# <span style="color:#F26835; font-size:34px;" >Logistic Regression</span>
###### <span style="color:#A1A1A1;"> Students: </span>
###### <span style="color:#A1A1A1;">Instructor: Magnús Eðvald Björnsson </span>

## Question
> ###### <span style="color:#555;">What is Regression ? </span>


Regression is a measure of the relation between the mean value of one variable (e.g., output) and corresponding values of other variables ( time and cost for example).

## Question
> ###### <span style="color:#555;">What is Logistic Regression ? </span>


Logistic Regression is a helpful tool, when we want to answer yes or no questions, i.e. when the outcome is binary.

Simple logistic regression is when we use only one factor of measurement to get to the binary answer.

Examples of simple logistic regression:
* Using only credit scores to answer the probability someone with a credit of 700 would get a loan.



Logistic Regression steps.
* 1. Model

You have a credit score that you feed into a decision model and the outcome is either: you get a loan or you get rejected.
###### How is the model made?

* 2. estimate
* 3. predict
* 4. classify


BLA--section

$ m = \dfrac{ \overline{xy} - \bar{x} \bar{y}} {x_2 - x_1} $

or psuedo-code
```
( meanof(x*y) - (mean_of(x)*mean_of(y)) ) / (mean_of(x^2) - mean_of(x)^2 )

```


## Logit function
A logit function is simply a function of the mean of the response variable Y that we use as the response instead of Y itself.

Because Y is a categorical binary variable and we need to predict in percentages, the logistic model uses the logit function to help us transform it into a response between 0 and 1

It uses the natural log function to form a linear line, for example if we display many outcomes(from 0 to 1) compared to different inputs on the other axis.

So the logit function is useful because it can take any real number as input and the output always takes a value between zero and one.

In the formula below: $\beta_{0} + \beta_{1}X_{1} $ is $\beta_{0} + \beta_{1}X_{1} + \beta_{2}X_{2} + \dotso + \beta_{k}X_{k } $ or the  linear compination of independent variables,  $\beta_{0} $ is the first and $\beta_{1}X_{1} $ is the rest.



$ Ln  \bigg( \dfrac{{P}} {1 - P} \bigg) = \beta_{0} + \beta_{1}X_{1} + \beta_{2}X_{2} + \dotso + \beta_{k}X_{k}  $ 


Then the antilog of the the logit function allows us to find estimated regression equation.

$ \dfrac{{P}} {1 - P} = e^{\beta_{0} + \beta_{1}X_{1} + \beta_{2}X_{2} + \dotso + \beta_{k}X_{k}}  $ 

$ P = e^{\beta_{0} + \beta_{1}X_{1} + \beta_{2}X_{2} + \dotso + \beta_{k}X_{k}} \big( {1 - P} \big) $ 


We end up with the estimated regression equation

$ \widehat{P} = \dfrac{e^{\beta_{0} + \beta_{1}X_{1}}} {1 + e^{\beta_{0} + \beta_{1}X_{1}} } $


# (Display that stuff)

# then inverse latex

# Because we want our probabilities on the y axis we can take the inverse of the logit 

# (Display that inverse logit)



Then we compute the inverse logit because we want our percentage as the Y variable / dependent. 

## Question 
> ###### <span style="color:#555;"> How is it calculated ? </span>


### <span style="color:#498BA6;">Answer </span>
##### Arranging the formula


$ \bar{x} \text{ and  } \bar{y}_1 \text{ blabla  }  $

#### <span style="color:#F2B33D;">Verifying Results:</span>

Asserting that blabla

### <span style="color:#B19B7D;"> A simple example calculated </span>

## Question
> ###### <span style="color:#555;">Why do we need Logistic Regression ? </span>


###### Predict the future

People often need to try and predict the future based on past events, the logistic regression model can help because it outputs the probability that an observation is in a particular category depending on the values of the input variables. 


To sum it up

logisitic regression does a couple of things:


#### We need to link together our independant variables to essentially; the Bernoulli distrubution. That link is called the logit
##the regression coefficients represent the change in the logit for each unit change in the predictor
# now hold on, we can't plot the line just yet, we have our individual credit scores and their categorical binary value.
# (display that scatter plot)
# First we are going to have to develop an estimated regression equation for our individual credit scores that fits the inverse logit model. To complete our special inverse logit model for these sample of credit scores we need to get the coefficients for "a" our analysis gives back to us, and put them into the estimated regression model and when we plot them we are going to get a linear shape that complements the credit score range.

the regression coefficients for logistic regression are calculated using maximum likelihood estimation or MLE


In [226]:
from bokeh.plotting import figure, show, ColumnDataSource, output_notebook
from bokeh.models.tools import HoverTool, BoxZoomTool, CrosshairTool, Tool
import numpy as np

output_notebook()

## Variables ##
urtak_size = 50

# Independent variables and their binary state
# THIS NEEDS TO BE CHANGED TO READING FROM A FILE
x_past_credit_scores = np.random.randint(100, 400, size = urtak_size).tolist()
y_zeroes_and_ones = np.random.randint(2, size = urtak_size).tolist()
print(type(x_past_credit_scores))
####################

print(x_past_credit_scores)
print(y_zeroes_and_ones)
binary_source = ColumnDataSource(
        data=dict(
            x=x_past_credit_scores,
            y=y_zeroes_and_ones
        )
    )


plot = figure(plot_width=1000, plot_height=400,
              y_axis_label = "Percentages",
              x_axis_label = "Credit score", 
              title="Credit Scores of "+ str(urtak_size) + " individuals on a scatter plot",
              x_range=(0,440))

plot.circle('x', 'y', size=20, source = binary_source)

show(plot)


Loading BokehJS ...

<class 'list'>
[120, 398, 165, 242, 104, 196, 214, 394, 132, 166, 337, 350, 133, 271, 392, 244, 377, 130, 253, 378, 379, 237, 335, 142, 151, 342, 167, 101, 376, 365, 362, 283, 313, 159, 181, 142, 251, 244, 209, 307, 264, 225, 331, 287, 184, 383, 312, 259, 230, 398]
[1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1]


In [228]:

p_x = np.arange(0.01,0.9999,0.01).tolist()

r_y = [np.log(i/(1-i)) for i in p_x]

stuff = (np.log(0.7)-np.log(1-0.7))
print(stuff)
#r_x, r_y = zip(*((np.log(i)-np.log(1-i), i) for i in regression))
#r_x, r_y = zip(*((i, np.exp(i)) for i in regression))
print(p_y)
print(r_y)



logit_source = ColumnDataSource(
        data=dict(
            x=p_x, #[ 0.01,  0.11,  0.21 , 0.31 , 0.41 , 0.51 , 0.61 , 0.71  ,0.81  ,0.91 ],
            y=r_y, #[-4.59511985, -2.0907411 , -1.32492541, -0.8001193 , -0.36396538, 0.04000533,  0.44731222,  0.89538405,  1.45001018,  2.31363493]
        )
    )

logit_plot = figure(plot_width=400, plot_height=400, 
              x_axis_label = "Percentages", 
              title="Logit function: Ln(P/1-P) - percentage on the X axis",
                x_range=(0,1),
                y_range=(-6,4)
            )
              
logit_plot.line('x','y', source = logit_source )

show(logit_plot)

0.847297860387
[0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.06999999999999999, 0.08, 0.09, 0.09999999999999999, 0.11, 0.12, 0.13, 0.14, 0.15000000000000002, 0.16, 0.17, 0.18000000000000002, 0.19, 0.2, 0.21000000000000002, 0.22, 0.23, 0.24000000000000002, 0.25, 0.26, 0.27, 0.28, 0.29000000000000004, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35000000000000003, 0.36000000000000004, 0.37, 0.38, 0.39, 0.4, 0.41000000000000003, 0.42000000000000004, 0.43, 0.44, 0.45, 0.46, 0.47000000000000003, 0.48000000000000004, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.5700000000000001, 0.5800000000000001, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.6900000000000001, 0.7000000000000001, 0.7100000000000001, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.8200000000000001, 0.8300000000000001, 0.8400000000000001, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.9400000000000001, 0.9500000000000001, 0.9600000000000001, 0.97, 0.98, 0.99]
[-4.5951198501345898, -3.8918202

In [229]:
from sklearn.linear_model import logistic
print(type(logistic._intercept_dot))

x_past_credit_scores = np.random.randint(100, 400, size = urtak_size)
y_zeroes_and_ones = np.random.randint(2, size = urtak_size)


w, y, z = logistic._intercept_dot(x_past_credit_scores,y_zeroes_and_ones)


<class 'function'>


In [353]:
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


x_past_credit_scores = np.random.randint(100, 400, size = urtak_size)
y_zeroes_and_ones = np.random.randint(2, size = urtak_size)
labels = ["accepted",'creditscore']
# read the data in
#df = pd.read_csv("http://www.ats.ucla.edu/stat/data/binary.csv")
input = list(zip(y_zeroes_and_ones,x_past_credit_scores))
df = pd.DataFrame.from_records(input,columns=labels)
print(df.head())

print()
print(df.accepted.value_counts())


   accepted  creditscore
0         0          272
1         0          391
2         1          387
3         0          257
4         0          386

0    28
1    22
Name: accepted, dtype: int64


In [362]:
# The Statsmodel function sm.Logit requires us to give an explicit intercept column beforehand

kp = 22 / 50
print(kp)
# intercept going in we want as 
odds_kp = kp/(1-kp)
print(odds_kp)
print(np.log(odds_kp))
df['intercept'] = odds_kp

# kp = 22/ 50
# intercept going in we want as 
# odds_kp = kp/1-kp

train_cols = df.columns[1:]
# Index(['creditscore', 'intercept'], dtype='object')

# the Logit uses the Newton optimizer in Maximum Likelihood Estimation (MLE), (see print(result.mle_settings) )

logit = sm.Logit(df['accepted'], df[train_cols])


result = logit.fit()
print(result.summary())
print(dir(result))
print()
print(result.mle_retvals)

print()

print(result.summary2())

print()

print(result.mle_settings)
print()

print(result)

print(np.log(0.3245))

0.44
0.7857142857142857
-0.241162056817
Optimization terminated successfully.
         Current function value: 0.685634
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:               accepted   No. Observations:                   50
Model:                          Logit   Df Residuals:                       48
Method:                           MLE   Df Model:                            1
Date:                Thu, 23 Feb 2017   Pseudo R-squ.:               0.0004313
Time:                        22:11:55   Log-Likelihood:                -34.282
converged:                       True   LL-Null:                       -34.296
                                        LLR p-value:                    0.8634
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
creditscore    -0.0006      0.003     -0.172      0.863        -0.

In [ ]:
from sklearn import datasets
import numpy as np
iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target

print(len(y))
print(len(X))


from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=3.0, random_state=0)
lr.fit(X_train_std, y_train)

# Decision region drawing
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):
   # setup marker generator and color map
   markers = ('s', 'x', 'o', '^', 'v')
   colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
   cmap = ListedColormap(colors[:len(np.unique(y))])

   # plot the decision surface
   x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
   x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
   xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
   np.arange(x2_min, x2_max, resolution))
   Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
   Z = Z.reshape(xx1.shape)
   plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
   plt.xlim(xx1.min(), xx1.max())
   plt.ylim(xx2.min(), xx2.max())

   # plot all samples
   X_test, y_test = X[test_idx, :], y[test_idx]
   for idx, cl in enumerate(np.unique(y)):
      plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
               alpha=0.8, c=cmap(idx),
               marker=markers[idx], label=cl)
   # highlight test samples
   if test_idx:
      X_test, y_test = X[test_idx, :], y[test_idx]
      plt.scatter(X_test[:, 0], X_test[:, 1], c='',
               alpha=1.0, linewidth=1, marker='o',
               s=55, label='test set')

X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

plot_decision_regions(X_combined_std,
                      y_combined, classifier=lr,
                      test_idx=range(105,150))

plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.show()